In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [ ]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span",attrs= {'id':'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product MRP
def get_mrp(soup):

    try:
        mrp = soup.find('span',attrs={'class':'a-price a-text-price'}).find('span',attrs= {'class':'a-offscreen'}).text.strip(u"\u20B9")

    except AttributeError:
        mrp = ""

    return mrp
# Function to extract Discount given on MRP
def get_discount(soup):
    
    try:
        discount = new_soup.find('span',attrs={'class':'a-size-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage'}).text.strip('-')
    
    except AttributeError:
        discount = ""
    
    return discount
        
# Function to extract Product Selling price
def get_price(soup):
    
    try:
        price = soup.find("span",attrs= {'class':'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'}).find("span",attrs= {'class':'a-offscreen'}).text.strip(u"\u20B9")
    
    except AttributeError:
        price = ''
        
    return price

# Function to extract Product Rating
def get_rating(soup):
    
    try:
        rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
    
    except AttributeError:
        rating = 'NA'

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).text.strip('ratings ')

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract EMI Status
def get_emi(soup):
    try:
        emi = new_soup.find('span',attrs={'class':'a-hidden'}).text.strip('EMI starts at u"\u20B9')
        #emi_value = emi_string.split()[0]
        
    except AttributeError:
        emi = "NA"

    return emi

In [ ]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=mobile+phone&ref=nb_sb_noss"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch title links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"Title":[], "MRP":[], "Discount":[], "Price":[],"Rating":[],"No.of Reviews":[],"EMI":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['Title'].append(get_title(new_soup))
        d['MRP'].append(get_mrp(new_soup))
        d['Discount'].append(get_discount(new_soup))
        d['Price'].append(get_price(new_soup))
        d['Rating'].append(get_rating(new_soup))
        d['No.of Reviews'].append(get_review_count(new_soup))
        d['EMI'].append(get_emi(new_soup))

    
    amazon_data = pd.DataFrame.from_dict(d)
    amazon_data.to_csv("Amazon_data_phone.csv", header=True, index=False)
